# Scikit-learn - Python 通用机器学习库教程

欢迎来到 Scikit-learn 教程！Scikit-learn (也写作 sklearn) 是 Python 中最流行、最全面的“经典”机器学习库。它提供了大量用于数据预处理、模型选择、模型训练、评估以及常见机器学习算法（分类、回归、聚类、降维）的工具。

**为什么 Scikit-learn 对 ML/DL/数据科学很重要？**

1.  **一致的 API**：提供了简单、一致的接口 (`fit`, `predict`, `transform`) 来使用不同的算法。
2.  **广泛的算法覆盖**：包含了绝大多数常用的非深度学习算法。
3.  **数据预处理工具**：提供了丰富的特征缩放、编码、缺失值处理等工具。
4.  **模型选择与评估**：内置交叉验证、超参数调优和各种评估指标。
5.  **与其他库集成良好**：与 NumPy, SciPy, Pandas, Matplotlib 等紧密集成。
6.  **优秀的文档和社区支持**：文档清晰，示例丰富，社区活跃。
7.  **学习基础**：Scikit-learn 的设计思想和 API 风格对许多其他机器学习库（甚至深度学习库的部分接口）产生了深远影响。

**本教程将涵盖 Scikit-learn 的核心工作流程和常用功能：**

1.  加载数据集
2.  数据预处理 (特征缩放, 编码)
3.  数据集划分 (训练集/测试集)
4.  模型训练 (`fit`)
5.  模型预测 (`predict`, `predict_proba`)
6.  常用模型示例 (分类: Logistic Regression, Random Forest; 回归: Linear Regression; 聚类: KMeans)
7.  模型评估 (常用指标)
8.  交叉验证 (`cross_val_score`)
9.  超参数调优 (`GridSearchCV`)
10. 管道 (`Pipeline`)

## 准备工作：导入必要的库

我们将导入 Scikit-learn 中需要的模块，以及 NumPy 和 Pandas。

In [ ]:
# Standard Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learn modules
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, 
                        mean_squared_error, r2_score, confusion_matrix, classification_report
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_iris, load_boston # Example datasets (load_boston deprecated, use fetch_california_housing or others)
from sklearn.compose import ColumnTransformer

# Set plotting style
sns.set_theme(style="whitegrid")

print("Libraries imported.")

# Handle potential deprecation of load_boston
try:
    from sklearn.datasets import fetch_california_housing
    california_housing = fetch_california_housing(as_frame=True) # Use as_frame=True to get a Pandas DataFrame
    print("Using California Housing dataset.")
    regression_data = california_housing
except ImportError:
    print("fetch_california_housing not available. Some regression examples might be limited.")
    regression_data = None

# Load Iris dataset for classification
iris = load_iris(as_frame=True)
iris_df = iris.data
iris_df['target'] = iris.target
iris_df['species'] = iris_df['target'].map({i: name for i, name in enumerate(iris.target_names)}) # Add species names
print("\nIris dataset loaded into DataFrame:")
print(iris_df.head())

## 2. 数据预处理

机器学习模型通常需要数值型、标准化的输入数据。预处理步骤包括处理缺失值、转换分类特征和缩放数值特征。

*   **特征缩放 (Scaling)**：将数值特征缩放到相似的范围，防止某些特征因数值范围过大而主导模型。
    *   `StandardScaler`: 标准化 (均值为0，方差为1)。
    *   `MinMaxScaler`: 归一化到 [0, 1] (或指定范围)。
*   **分类特征编码 (Encoding)**：将文本或类别标签转换为数值表示。
    *   `LabelEncoder`: 将标签编码为 0 到 n_classes-1 的整数 (通常用于目标变量)。
    *   `OneHotEncoder`: 将具有 k 个类别的特征转换为 k 个二元 (0/1) 特征 (通常用于输入特征，避免引入错误的顺序关系)。
*   **缺失值处理 (Imputation)**：用特定策略（如均值、中位数、众数）填充缺失值 (`NaN`)。
    *   `SimpleImputer`

In [ ]:
print("--- Data Preprocessing Examples ---")

# --- Feature Scaling ---
print("\n--- Feature Scaling ---")
data_scale = np.array([[1., -1., 2.], [2., 0., 0.], [0., 1., -1.]])
print(f"Original data:\n{data_scale}")

scaler_standard = StandardScaler()
scaled_standard = scaler_standard.fit_transform(data_scale)
print(f"\nStandardized data (mean ~0, std ~1):\n{scaled_standard}")
print(f"Mean after scaling: {scaled_standard.mean(axis=0)}")
print(f"Std after scaling: {scaled_standard.std(axis=0)}")

scaler_minmax = MinMaxScaler()
scaled_minmax = scaler_minmax.fit_transform(data_scale)
print(f"\nMinMax scaled data (range [0, 1]):\n{scaled_minmax}")

# --- Categorical Encoding ---
print("\n--- Categorical Encoding ---")
categorical_feature = [['Male', 'Low'], ['Female', 'Medium'], ['Female', 'High'], ['Male', 'Low']]
df_cat = pd.DataFrame(categorical_feature, columns=['Gender', 'Level'])
print(f"Original categorical data:\n{df_cat}")

# OneHotEncoder for input features
onehot_encoder = OneHotEncoder(sparse_output=False) # sparse=False returns numpy array
encoded_features = onehot_encoder.fit_transform(df_cat)
print(f"\nOneHot encoded features:\n{encoded_features}")
print(f"Feature names: {onehot_encoder.get_feature_names_out()}")

# LabelEncoder for target variable (example)
target_labels = ['Cat', 'Dog', 'Cat', 'Fish', 'Dog']
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(target_labels)
print(f"\nOriginal labels: {target_labels}")
print(f"Label encoded labels: {encoded_labels}")
print(f"Encoded classes: {label_encoder.classes_}")

# --- Imputation --- 
print("\n--- Imputation (Missing Values) ---")
data_missing = np.array([[1, 2, np.nan], [4, np.nan, 6], [7, 8, 9]])
print(f"Data with missing values:\n{data_missing}")

imputer_mean = SimpleImputer(strategy='mean')
imputed_data = imputer_mean.fit_transform(data_missing)
print(f"\nImputed data (using mean):\n{imputed_data}")

## 3. 数据集划分 (训练集/测试集)

将数据集分为训练集和测试集是评估模型泛化能力的关键步骤。
`train_test_split` 是常用的工具。

In [ ]:
print("--- Train/Test Split --- ")
# 使用 Iris 数据集
X = iris_df[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']] # 特征
y = iris_df['target'] # 目标变量 (类别标签 0, 1, 2)

# test_size: 测试集比例或数量
# random_state: 随机种子，确保每次划分结果一致
# stratify=y: 对于分类问题，确保训练集和测试集中各类别比例与原始数据一致
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

print(f"Original dataset shape: X={X.shape}, y={y.shape}")
print(f"Training set shape: X_train={X_train.shape}, y_train={y_train.shape}")
print(f"Test set shape: X_test={X_test.shape}, y_test={y_test.shape}")
print(f"\nProportion of classes in original y:\n{y.value_counts(normalize=True)}")
print(f"\nProportion of classes in y_train:\n{y_train.value_counts(normalize=True)}")
print(f"\nProportion of classes in y_test:\n{y_test.value_counts(normalize=True)}")

## 4. 模型训练 (`fit`)

Scikit-learn 的核心 API 非常一致：
1.  选择一个模型类并实例化。
2.  使用训练数据调用实例的 `fit(X_train, y_train)` 方法。

In [ ]:
print("--- Model Training Example (Logistic Regression) ---")

# 0. (Optional but often needed) Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) # Fit on training data, then transform
# IMPORTANT: Use the SAME scaler fitted on training data to transform the test data
X_test_scaled = scaler.transform(X_test)
print("Features scaled using StandardScaler.")

# 1. Instantiate the model
log_reg = LogisticRegression(random_state=42, max_iter=200) # Increase max_iter if needed
print(f"Model instantiated: {log_reg}")

# 2. Train the model using scaled training data
log_reg.fit(X_train_scaled, y_train)
print("Model training completed (fitted).")

# 模型已训练完成，其内部参数（如系数、截距）已被学习

## 5. 模型预测 (`predict`, `predict_proba`)

训练好的模型可以用来对新数据（通常是测试集）进行预测。
*   `predict(X_test)`: 对 `X_test` 中的每个样本预测类别标签（分类）或数值（回归）。
*   `predict_proba(X_test)`: （仅限分类模型）返回每个样本属于各个类别的概率。

In [ ]:
print("--- Model Prediction --- ")

# 使用训练好的 log_reg 模型和缩放后的测试数据 X_test_scaled
y_pred = log_reg.predict(X_test_scaled)
print(f"Predicted labels for test set (first 10): {y_pred[:10]}")
print(f"Actual labels for test set (first 10):   {y_test.values[:10]}")

# 获取预测概率
y_pred_proba = log_reg.predict_proba(X_test_scaled)
print(f"\nPredicted probabilities for test set (first 5 samples):\n{y_pred_proba[:5].round(3)}")
# 每一行对应一个样本，每一列对应一个类别的概率 (顺序由 model.classes_ 决定)
print(f"Model classes: {log_reg.classes_}")

## 6. 常用模型示例

Scikit-learn 提供了多种模型。

In [ ]:
print("--- Other Model Examples ---")

# --- Random Forest Classifier ---
print("\nTraining Random Forest Classifier...")
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1) # n_jobs=-1 使用所有CPU核心
rf_clf.fit(X_train_scaled, y_train)
y_pred_rf = rf_clf.predict(X_test_scaled)
print("Random Forest training complete.")
# Evaluation will be done later

# --- Support Vector Classifier (SVC) ---
print("\nTraining Support Vector Classifier...")
svc_clf = SVC(probability=True, random_state=42) # probability=True enables predict_proba
svc_clf.fit(X_train_scaled, y_train)
y_pred_svc = svc_clf.predict(X_test_scaled)
print("SVC training complete.")

# --- Linear Regression (using California Housing) ---
if regression_data is not None:
    print("\n--- Linear Regression Example --- ")
    X_reg = regression_data.data
    y_reg = regression_data.target
    
    X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(X_reg, y_reg, test_size=0.3, random_state=42)
    
    # Scale regression features
    reg_scaler = StandardScaler()
    X_reg_train_scaled = reg_scaler.fit_transform(X_reg_train)
    X_reg_test_scaled = reg_scaler.transform(X_reg_test)
    
    lin_reg = LinearRegression()
    lin_reg.fit(X_reg_train_scaled, y_reg_train)
    y_pred_reg = lin_reg.predict(X_reg_test_scaled)
    print("Linear Regression training complete.")
    print(f"First 5 regression predictions: {y_pred_reg[:5].round(2)}")
    print(f"First 5 actual regression values: {y_reg_test.values[:5].round(2)}")
else:
    print("\nSkipping Linear Regression example (dataset not loaded).")

# --- K-Means Clustering (Unsupervised) ---
print("\n--- K-Means Clustering Example --- ")
# 使用未标记的 Iris 特征 (假设我们不知道类别)
X_iris_unscaled = X # Use original unscaled data for clustering interpretation, or scale it
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10) # n_clusters 通常需要预先确定或尝试不同值; n_init for stability
kmeans.fit(X_iris_unscaled) # 无监督学习，只需要 X
cluster_labels = kmeans.labels_
print(f"K-Means training complete. Cluster labels (first 20): {cluster_labels[:20]}")
print(f"Cluster centers (centroids):\n{kmeans.cluster_centers_.round(2)}")

# 比较 K-Means 找到的簇与真实类别 (需要注意簇标签与真实标签可能不对应)
df_cluster_comparison = pd.DataFrame({'TrueLabel': y, 'ClusterLabel': cluster_labels})
print("\nCross-tabulation of True Labels vs K-Means Clusters:")
print(pd.crosstab(df_cluster_comparison['TrueLabel'], df_cluster_comparison['ClusterLabel']))

## 7. 模型评估

评估模型在测试集上的性能至关重要。

**分类常用指标:**
*   `accuracy_score`: 准确率 (正确预测的比例)。
*   `precision_score`: 精确率 (预测为正的样本中，实际为正的比例)。
*   `recall_score`: 召回率 (实际为正的样本中，被正确预测为正的比例)。
*   `f1_score`: F1 分数 (精确率和召回率的调和平均数)。
*   `confusion_matrix`: 混淆矩阵。
*   `classification_report`: 包含精确率、召回率、F1 分数的文本报告。
*   `roc_auc_score`: ROC 曲线下面积 (适用于二分类或多分类的 OvR/OvO)。

**回归常用指标:**
*   `mean_squared_error (MSE)`: 均方误差。
*   `mean_absolute_error (MAE)`: 平均绝对误差。
*   `r2_score`: R 方（决定系数），表示模型解释了多少因变量的方差。

In [ ]:
print("--- Model Evaluation ---")

# --- Classification Evaluation (using Logistic Regression results) ---
print("\n--- Logistic Regression Evaluation ---")
accuracy_logreg = accuracy_score(y_test, y_pred)
# For multiclass precision/recall/f1, need 'average' parameter (e.g., 'macro', 'micro', 'weighted')
precision_logreg = precision_score(y_test, y_pred, average='weighted')
recall_logreg = recall_score(y_test, y_pred, average='weighted')
f1_logreg = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy_logreg:.4f}")
print(f"Precision (weighted): {precision_logreg:.4f}")
print(f"Recall (weighted): {recall_logreg:.4f}")
print(f"F1 Score (weighted): {f1_logreg:.4f}")

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=iris.target_names))

# ROC AUC (requires probabilities, use OvR for multiclass)
# roc_auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovr', average='weighted')
# print(f"ROC AUC (weighted OvR): {roc_auc:.4f}")

# --- Regression Evaluation (using Linear Regression results) ---
if regression_data is not None:
    print("\n--- Linear Regression Evaluation ---")
    mse_linreg = mean_squared_error(y_reg_test, y_pred_reg)
    r2_linreg = r2_score(y_reg_test, y_pred_reg)
    print(f"Mean Squared Error (MSE): {mse_linreg:.4f}")
    print(f"R-squared (R2): {r2_linreg:.4f}")
else:
    print("\nSkipping Regression Evaluation.")

## 8. 交叉验证 (`cross_val_score`)

简单的训练/测试集划分可能因划分的随机性导致评估结果不稳定。交叉验证通过将数据分成多个“折”(fold)，轮流使用其中一折作为验证集，其余作为训练集，然后对多次评估结果取平均，来提供更稳健的模型性能估计。

`cross_val_score` 是一个便捷的函数。

In [ ]:
print("--- Cross-Validation Example (Random Forest on Iris) ---")

# 使用完整的、缩放后的 Iris 数据集进行交叉验证
X_iris_scaled = scaler.fit_transform(X) # Scale the full dataset
y_iris = y

rf_clf_cv = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

# cv 参数指定折数 (e.g., 5-fold cross-validation)
# scoring 参数指定评估指标 (e.g., 'accuracy', 'f1_weighted', 'neg_mean_squared_error' for regression)
cv_scores = cross_val_score(rf_clf_cv, X_iris_scaled, y_iris, cv=5, scoring='accuracy')

print(f"Cross-validation scores (accuracy): {cv_scores}")
print(f"Average CV accuracy: {cv_scores.mean():.4f}")
print(f"Standard deviation of CV accuracy: {cv_scores.std():.4f}")

## 9. 超参数调优 (`GridSearchCV`)

机器学习模型通常有许多**超参数**（在训练前设置的参数，如 Random Forest 的 `n_estimators` 或 SVC 的 `C` 和 `gamma`），它们会影响模型性能。

`GridSearchCV` 通过系统地尝试超参数网格中的所有组合，并使用交叉验证来评估每种组合的性能，从而找到最佳的超参数设置。

In [ ]:
print("--- Hyperparameter Tuning with GridSearchCV (SVC on Iris) ---")

# 定义要搜索的参数网格
param_grid = {
    'C': [0.1, 1, 10, 100],           # 正则化参数
    'gamma': [1, 0.1, 0.01, 0.001],    # RBF 核的系数 ('rbf'是默认核)
    'kernel': ['rbf', 'linear']       # 尝试不同的核函数
}

# 创建 GridSearchCV 对象
# estimator: 要调优的模型实例
# param_grid: 参数网格
# cv: 交叉验证折数
# scoring: 评估指标
# n_jobs=-1: 使用所有CPU核心
grid_search = GridSearchCV(SVC(random_state=42), 
                           param_grid, 
                           cv=5, 
                           scoring='accuracy', 
                           n_jobs=-1, 
                           verbose=1) # verbose 控制输出信息的详细程度

# 在训练数据上执行网格搜索 (它会自动进行交叉验证)
print("Starting Grid Search...")
grid_search.fit(X_train_scaled, y_train) # Fit on the training set
print("Grid Search complete.")

# 查看最佳参数和最佳分数
print(f"\nBest parameters found: {grid_search.best_params_}")
print(f"Best cross-validation accuracy: {grid_search.best_score_:.4f}")

# 获取最佳模型
best_svc = grid_search.best_estimator_

# 使用最佳模型在测试集上评估
y_pred_best_svc = best_svc.predict(X_test_scaled)
accuracy_best_svc = accuracy_score(y_test, y_pred_best_svc)
print(f"\nAccuracy on test set with best SVC: {accuracy_best_svc:.4f}")

## 10. 管道 (`Pipeline`)

管道 (`Pipeline`) 可以将多个数据处理步骤（如缩放、特征选择）和一个最终的模型估计器链接在一起。这有几个好处：
*   **方便性**：只需对管道调用一次 `fit` 和 `predict`。
*   **防止数据泄露**：确保预处理步骤（如缩放）只在训练数据上 `fit`，然后应用于测试数据，这在交叉验证和网格搜索中尤其重要。
*   **代码简洁**：将工作流封装在一个对象中。

In [ ]:
print("--- Pipeline Example (Scaling + SVC) ---")

# 创建管道：包含一个缩放器和一个分类器
pipeline = Pipeline([
    ('scaler', StandardScaler()), # 步骤1：命名为 'scaler'，使用 StandardScaler
    ('svc', SVC(random_state=42))  # 步骤2：命名为 'svc'，使用 SVC
])

# 直接在原始训练数据上训练管道
# 管道会自动处理：
# 1. scaler.fit_transform(X_train)
# 2. svc.fit(scaled_X_train, y_train)
pipeline.fit(X_train, y_train) 
print("Pipeline trained.")

# 使用管道进行预测
# 管道会自动处理：
# 1. scaler.transform(X_test) (使用在训练集上fit的scaler)
# 2. svc.predict(scaled_X_test)
y_pred_pipeline = pipeline.predict(X_test)
accuracy_pipeline = accuracy_score(y_test, y_pred_pipeline)
print(f"Accuracy using pipeline on test set: {accuracy_pipeline:.4f}")

# 管道也可以与 GridSearchCV 结合使用，调优模型参数甚至预处理步骤的参数
print("\n--- Pipeline with GridSearchCV ---")
param_grid_pipeline = {
    'svc__C': [0.1, 1, 10],        # 注意参数名前缀：步骤名 + 双下划线 + 参数名
    'svc__gamma': [0.1, 0.01]
}

grid_search_pipe = GridSearchCV(pipeline, param_grid_pipeline, cv=3, scoring='accuracy', verbose=0)
print("Starting Grid Search with Pipeline...")
grid_search_pipe.fit(X_train, y_train) # Fit on original X_train
print("Grid Search with Pipeline complete.")
print(f"Best params for pipeline: {grid_search_pipe.best_params_}")
print(f"Best CV score for pipeline: {grid_search_pipe.best_score_:.4f}")

# 评估最佳管道
best_pipeline = grid_search_pipe.best_estimator_
y_pred_best_pipe = best_pipeline.predict(X_test)
accuracy_best_pipe = accuracy_score(y_test, y_pred_best_pipe)
print(f"Accuracy on test set with best pipeline: {accuracy_best_pipe:.4f}")

## 总结

Scikit-learn 是 Python 中进行经典机器学习任务的强大而全面的库。其一致的 API、丰富的算法和工具集，使其成为数据科学家和机器学习工程师的必备技能。

**关键要点：**
*   遵循**数据加载 -> 预处理 -> 划分 -> 训练 -> 预测 -> 评估**的基本流程。
*   熟练使用 `StandardScaler`, `OneHotEncoder` 等预处理工具。
*   掌握 `fit()`, `predict()`, `transform()` 核心方法。
*   了解常用分类、回归、聚类算法的 Scikit-learn 实现。
*   使用交叉验证和网格搜索进行稳健的模型评估和超参数调优。
*   利用 `Pipeline` 简化工作流程并避免数据泄露。

Scikit-learn 的功能远不止于此，还包括特征选择、降维 (PCA, t-SNE)、更复杂的模型集成、半监督学习等。强烈建议深入学习其官方文档和用户指南。